In [1]:
#!pip install opendatasets
!pip install -U albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.2.1
    Uninstalling albumentations-1.2.1:
      Successfully uninstalled albumentations-1.2.1


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mlops/hw2

Mounted at /content/drive
/content/drive/MyDrive/mlops/hw2


In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from pandas.core.common import flatten
import copy
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import glob
from tqdm import tqdm

## import library
import numpy as np
import os
import random
import sys
from pycocotools.coco import COCO
import urllib
import zipfile
import shutil
from google.colab.patches import cv2_imshow
from skimage import io
from PIL import Image
#import opendatasets as od
import pandas

In [4]:
imgs_path = ['/content/drive/MyDrive/mlops/hw2/chest-xray-pneumonia/chest_xray/train/',
                '/content/drive/MyDrive/mlops/hw2/chest-xray-pneumonia/chest_xray/test/',
                '/content/drive/MyDrive/mlops/hw2/chest-xray-pneumonia/chest_xray/val/']

file_list_ = []
for img_path in imgs_path:
 
  file_list = glob.glob(img_path + "*")
  for i in file_list: 
    file_list_.append(i)

data = []

for class_path in file_list_:
            class_name = class_path.split("/")[-1]
            filenames = [f for f in os.listdir(class_path) if f.endswith(('jpg', 'jpeg', 'png'))]
            for img_path in filenames:
                data.append([class_path+"/"+img_path, class_name])
print("number of all data:",len(data))


number of all data: 5856


In [5]:
from sklearn.model_selection import train_test_split

# Split data into 80% train and 20% test
train, test = train_test_split(data, test_size=0.2)

val, test = train_test_split(test, test_size=0.5)


# train : 4684
# val : 586
# test : 586


In [6]:
import torchvision.transforms as T
class CustomDataset(Dataset):
    def __init__(self,data, transform):
        self.data = data
        self.class_map = {"NORMAL" : 0, "PNEUMONIA": 1}
        self.img_dim = (224, 224)

    def rotateImage(image, angle):
      row,col = (228,228)
      center=tuple(np.array([row,col])/2)
      rot_mat = cv2.getRotationMatrix2D(center,angle,1.0)
      new_image = cv2.warpAffine(image, rot_mat, (col,row))
      return new_image        
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = Image.open(img_path).convert('RGB')
        img = cv2.imread(img_path)
        img = cv2.resize(img, self.img_dim)
        transform = T.RandomHorizontalFlip()
        if transform is True:
          self.img = rotateImage(img ,random.randint(0, 30))
          self.img = cv2.normalize(img, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
          img = transform(img)
        
        else:
          self.img = cv2.normalize(img, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        img_tensor = torch.from_numpy(img)
        class_id = self.class_map[class_name]
        '''For this conversion we use the permute function of torch,
            that allows us to change the ordering of the dimensions of a torch tensor. 
            The arguments we pass to it, correspond to the new ordering of dimensions we want.
            For example in our case, we have (Width, Height, Channels).
            (Width -> 0), (Height->1), (Channels->2)
              We want to reorder these dimensions to make channels first, therefore,
              we use img_tensor.permute(2, 0, 1), which would make the 2nd dimension first.'''
        img_tensor = img_tensor.permute(2, 0, 1)
        class_id = torch.tensor([class_id])
        return img_tensor, class_id


     

In [7]:
batch_size = 16

dataset = CustomDataset(train,transform = True )		
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset = CustomDataset(val ,transform = False)		
val_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset = CustomDataset(test,transform = False)
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [8]:
!pip install wandb -Uq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00


In [ ]:

import wandb
wandb.login()

sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric


parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'learning_rate': {
        # a flat distribution between 0 and 0.2
        'distribution': 'uniform',
        'min': 0,
        'max': 0.2
      },
    }

sweep_config['parameters'] = parameters_dict

import pprint

pprint.pprint(sweep_config)

sweep_id = wandb.sweep(sweep_config, project="MLops-HW2_1")

In [ ]:
from torchvision import models
from sklearn.metrics import accuracy_score
def build_network():
    # Load a pre-trained 
    model = models.resnet18(pretrained=True)

    # Freeze all layers in the pre-trained 
    for param in model.parameters():
        param.requires_grad = False

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 1)
    
    criterion = nn.BCELoss()

    return model.to(device),  criterion.to(device)
        
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.fc.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.fc.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(model, criterion, train_loader,val_loader, optimizer):
    running_loss = 0.0
    y_true = []
    y_pred = []
    for inputs, labels in train_loader:
        x = nn.Sigmoid()
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.float())
        outputs = x(outputs)
        loss = criterion(outputs, labels.float())

        running_loss += loss.item()

        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

        #epoch_loss = running_loss / len(train_loader)
    
    epoch_loss = running_loss / len(train_loader)
    print('epoch_loss: ', epoch_loss)

    model.eval() 

    val_loss = 0.0
    
    with torch.no_grad():
      for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs.float())
        outputs = x(outputs)
        loss = criterion(outputs, labels.float())
        val_loss += loss.item()

        pred =torch.round(outputs.detach())
        target = torch.round(labels.detach())
        

        y_pred.extend(pred.tolist())
        y_true.extend(target.tolist())

    val_acc = accuracy_score(y_true,y_pred)  
    val_loss = val_loss / len(val_loader)
    print(f'Train Loss: {epoch_loss:.4f}\t Val Loss: {val_loss:.4f}\t Val Acc: {val_acc:.4f}')
    print(f'val Loss: {epoch_loss:.4f}\t Val Loss: {val_loss:.4f}\t Val Acc: {val_loss:.4f}')
    print(f'val acc: {epoch_loss:.4f}\t Val Loss: {val_loss:.4f}\t Val Acc: {val_acc:.4f}')
    
    return epoch_loss ,val_loss ,val_acc

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn

def train(train_loader,val_loader,config=None):
    
    with wandb.init(config=config):
        
        config = wandb.config
        
        train_loader = train_loader
        val_loader = val_loader
        network, criterion = build_network()
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)
        num_epochs = 15
        for epoch in range(num_epochs):
            avg_loss,val_loss ,val_acc = train_epoch(network, criterion, train_loader,val_loader, optimizer)
            wandb.log({"loss_train": avg_loss, "epoch": epoch})     
            wandb.log({"loss_val": val_loss, "epoch": epoch})    
            wandb.log({"acc_val": val_acc, "epoch": epoch})          

In [ ]:
wandb.agent(sweep_id, train(train_loader,val_loader), count=5)